In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats.stats import pearsonr
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
# https://www.kaggle.com/code/fuzzywizard/rec-sys-collaborative-filtering-dl-techniques

C:\Users\system 4\AppData\Local\Temp\ipykernel_12832\1035269855.py:5: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
%cd Z:\PPMI_Data\Excels\CollaborativeFiltering
df = pd.read_csv('Feats45_unCategAge_APPRDX.csv')
df.shape
df.head(2)

Z:\PPMI_Data\Excels\CollaborativeFiltering


,PATNO,Patient_ID,APPRDX,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,3002,1,1,67.578082,1,0,12.16,7,1,1,...,57,1,0,45,59,47.5,110,204,17,1
1,3010,2,1,46.965716,1,1,7.76,6,1,1,...,36,1,3,40,39,52.0,27,52,9,0


In [3]:
# Removing a column
df1 = df.drop(['PATNO','APPRDX'], axis = 1)
df1.head(2)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,1,67.578082,1,0,12.16,7,1,1,1,1,...,57,1,0,45,59,47.5,110,204,17,1
1,2,46.965716,1,1,7.76,6,1,1,0,0,...,36,1,3,40,39,52.0,27,52,9,0


In [4]:
df2 = pd.read_csv('NonMotor_Empty.csv')
df2.shape
df2.head(2)

,Patient_ID,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Removing a column
df3 = df2.drop('Patient_ID', axis = 1)
df3.head(2)

,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,Education,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Ask the user for the updated value of age
new_age = int(input("What is the user's age (in years): "))

# Update the value of age in the dataframe
df3.at[0, 'Age'] = new_age

# Drop the 46th column from the original DataFrame
col_age = df3.pop('Age')

# Insert the column at the 5th position
df3.insert(0, "Age", col_age)
df3.head(2)

What is the user's age (in years): 54


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,54.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Below will take at least 3 features

In [8]:
def update_values(df3):
    # Input from the user for number of columns
    num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    while num_columns < 3 or num_columns > 45:
        print("Invalid number of columns. Please enter a number between 3 and 45.")
        num_columns = int(input("Enter the number of columns you want to update (minimum 3, maximum 45): "))
    
    columns = []
    values = []
     
     # Loop to get input from the user for each column
    for i in range(num_columns):
        column = input("Enter the name of the Feature %d: " % (i + 1))
        value = input("Enter the score of the Feature %d: " % (i + 1))
        
        columns.append(column)
        values.append(int(value))
    
    # Update the values in the dataframe
    for i in range(num_columns):
        df3.loc[df3[columns[i]] != values[i], columns[i]] = values[i]
    
    return df3

# Call the function
df3 = update_values(df3)

# Print the updated dataframe
df3.head()

Enter the number of columns you want to update (minimum 3, maximum 45): 3
Enter the name of the Feature 1: UPSIT
Enter the score of the Feature 1: 39
Enter the name of the Feature 2: Clock
Enter the score of the Feature 2: 3
Enter the name of the Feature 3: Apathy
Enter the score of the Feature 3: 2


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,54.0,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN


In [9]:
# Adding back the column (as the first column)
df3.insert (0, 'Patient_ID', df2['Patient_ID'])
df3.head()

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,999,54.0,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN


In [10]:
# merging two dataframes [original and our input dataframes]
merged_df = pd.concat ([df1, df3], axis = 0)
merged_df.reset_index(inplace = True)
merged_df.tail()

,index,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
247,247,248,57.447728,0.0,0.0,14.77,6.0,0.0,1.0,3.0,...,47.0,0.0,0.0,46.0,44.0,56.250000,27.0,75.0,36.0,0.0
248,248,249,58.260371,0.0,0.0,14.18,6.0,0.0,1.0,0.0,...,58.0,1.0,3.0,47.0,40.0,58.750000,28.0,48.0,38.0,1.0
249,249,250,66.383562,0.0,0.0,11.06,7.0,0.0,1.0,0.0,...,49.0,2.0,1.0,45.0,46.0,45.830002,36.0,106.0,32.0,1.0
250,250,251,68.073973,0.0,0.0,11.06,7.0,0.0,1.0,0.0,...,57.0,0.0,3.0,44.0,45.0,50.000000,30.0,80.0,38.0,0.0
251,0,999,54.000000,NaN,2.0,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN


In [12]:
# dropping a few columns
merged_df1 = merged_df.drop(['index','Patient_ID'], axis = 1)
merged_df1.head(2)


,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,67.578082,1.0,0.0,12.16,7.0,1.0,1.0,1.0,1.0,0.0,...,57.0,1.0,0.0,45.0,59.0,47.5,110.0,204.0,17.0,1.0
1,46.965716,1.0,1.0,7.76,6.0,1.0,1.0,0.0,0.0,0.0,...,36.0,1.0,3.0,40.0,39.0,52.0,27.0,52.0,9.0,0.0


In [13]:
# scaling values of every column from 1 to 5 [column wise]
scaler = MinMaxScaler (feature_range = (1,5))
df_scaled = scaler.fit_transform(merged_df1)
df_scaled = pd.DataFrame (df_scaled, columns = merged_df1.columns)
df_scaled.head(2)

,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,DopaDefic,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,3.786116,2.333333,1.0,3.503546,5.000000,3.0,1.0,2.333333,2.0,1.0,...,3.285714,3.0,1.0,3.434783,5.0,2.833333,3.832117,3.686131,2.333333,2.0
1,2.232411,2.333333,3.0,1.943262,4.333333,3.0,1.0,1.000000,1.0,1.0,...,1.785714,3.0,4.0,2.565217,3.0,3.133333,1.408759,1.467153,1.363636,1.0


In [14]:
# adding the column [as the first column]
df_scaled.insert (0, 'Patient_ID', merged_df['Patient_ID'], True)
df_scaled.head(3)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,1,3.786116,2.333333,1.0,3.503546,5.000000,3.0,1.0,2.333333,2.0,...,3.285714,3.0,1.0,3.434783,5.0,2.833333,3.832117,3.686131,2.333333,2.0
1,2,2.232411,2.333333,3.0,1.943262,4.333333,3.0,1.0,1.000000,1.0,...,1.785714,3.0,4.0,2.565217,3.0,3.133333,1.408759,1.467153,1.363636,1.0
2,3,4.267435,1.000000,1.0,3.975177,5.000000,3.0,5.0,1.000000,1.0,...,2.428571,3.0,3.0,2.913043,3.3,2.555333,3.248175,4.386861,3.424242,1.0


In [15]:
df_scaled.tail(3)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
249,250,3.696076,1.0,1.0,3.113475,5.000000,1.0,1.0,1.0,1.0,...,2.714286,5.0,2.0,3.434783,3.7,2.722,1.671533,2.255474,4.151515,2.0
250,251,3.823494,1.0,1.0,3.113475,5.000000,1.0,1.0,1.0,2.0,...,3.285714,1.0,4.0,3.260870,3.6,3.000,1.496350,1.875912,4.878788,1.0
251,999,2.762636,NaN,5.0,NaN,2.333333,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN


In [16]:
# using melt function [reshape]
df_melted = df_scaled.melt(id_vars='Patient_ID', var_name='Feature', value_name='Value')
print(df_melted.shape)
df_melted.head(2)

(11592, 3)


,Patient_ID,Feature,Value
0,1,Age,3.786116
1,2,Age,2.232411


In [17]:
#1. take groupby with respect to Feature and get mean avg of each Feature. 
feat_rating_count = df_melted.groupby('Patient_ID')['Value'].count()

#2. convert into dataframe for better operation work.
feat_rating_count = pd.DataFrame(feat_rating_count)

#3. change column name.
feat_rating_count.columns = ['Value_count']

#4. create new column Patient_ID
feat_rating_count['Patient_ID'] = feat_rating_count.index
feat_rating_count = feat_rating_count.reset_index(drop=True)

In [18]:
#merging to Rating dataframe so we have collectively all information together..
dframe = df_melted.merge(feat_rating_count, on = 'Patient_ID', how = 'inner')
print(dframe.shape)
dframe.head(2)

(11592, 4)


,Patient_ID,Feature,Value,Value_count
0,1,Age,3.786116,46
1,1,Anxiety,2.333333,46


In [19]:
#create a pivot matrix for user and Feature based on rating value..
RatingMat = dframe.pivot_table(index=['Patient_ID'],columns=['Feature'],values=['Value'],fill_value=0)

#keep a copy of original matrix for future refrence..
Original_RatingMat = RatingMat.copy()
RatingMat.head()

Value                                                          \
Feature          Age   Anxiety Apathy    Benton COGSTATE     Clock Cognition   
Patient_ID                                                                     
1           3.786116  2.333333      1  3.503546        1  5.000000         3   
2           2.232411  2.333333      3  1.943262        1  4.333333         3   
3           4.267435  1.000000      1  3.975177        5  5.000000         3   
4           3.521970  1.000000      1  2.659574        5  4.333333         3   
5           4.052632  2.333333      1  3.503546        1  5.000000         3   

                                         ...                            \
Feature    Constipate Depress DopaDefic  ...     STAIA STAIS  Semantic   
Patient_ID                               ...                             
1            2.333333       2         1  ...  3.434783   5.0  3.285714   
2            1.000000       1         1  ...  2.565217   3.0  1.785714   
3            1.000000       1         1  ...  2.913043   3.3  2.428571   
4            1.000000       1         1  ...  4.130435   3.8  2.714286   
5            1.000000       2         1  ...  3.956522   3.7  3.357143   

                                                                           \
Feature    SleepDay SleepNight Symbol_Digit Trail_Making_A Trail_Making_B   
Patient_ID                                                                  
1                 3          1     2.833333       3.832117       3.686131   
2                 3          4     3.133333       1.408759       1.467153   
3                 3          3     2.555333       3.248175       4.386861   
4                 3          3     2.500000       2.576642       3.116788   
5                 5          1     3.200000       1.817518       2.547445   

                            
Feature        UPSIT Urine  
Patient_ID                  
1           2.333333     2  
2           1.363636     1  
3           3.424242     1  
4           1.242424     2  
5           3.181818     1  

[5 rows x 46 columns]

In [20]:
#drop multilevel column for better usabiltity...
RatingMat.columns = RatingMat.columns.droplevel()
RatingMat.head(5)

Feature,Age,Anxiety,Apathy,Benton,COGSTATE,Clock,Cognition,Constipate,Depress,DopaDefic,...,STAIA,STAIS,Semantic,SleepDay,SleepNight,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,3.786116,2.333333,1,3.503546,1,5.000000,3,2.333333,2,1,...,3.434783,5.0,3.285714,3,1,2.833333,3.832117,3.686131,2.333333,2
2,2.232411,2.333333,3,1.943262,1,4.333333,3,1.000000,1,1,...,2.565217,3.0,1.785714,3,4,3.133333,1.408759,1.467153,1.363636,1
3,4.267435,1.000000,1,3.975177,5,5.000000,3,1.000000,1,1,...,2.913043,3.3,2.428571,3,3,2.555333,3.248175,4.386861,3.424242,1
4,3.521970,1.000000,1,2.659574,5,4.333333,3,1.000000,1,1,...,4.130435,3.8,2.714286,3,3,2.500000,2.576642,3.116788,1.242424,2
5,4.052632,2.333333,1,3.503546,1,5.000000,3,1.000000,2,1,...,3.956522,3.7,3.357143,5,1,3.200000,1.817518,2.547445,3.181818,1


In [21]:
#user wise similarity with cosine...
user_similarity = cosine_similarity(RatingMat)
user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
user_sim_df.head(5)

#we can do it using pearson correlation as well there is no hard and fast rule.
# generally user-user approach we use pearson correlation and item-item we use cosine similarity


#user wise similarity with pearson coorelation...
# user_similarity = 1-pairwise_distances(RatingMat, metric="correlation")
# user_sim_df = pd.DataFrame(user_similarity,index=RatingMat.index,columns=RatingMat.index)
# user_sim_df.head(5)

Patient_ID,1,2,3,4,5,6,7,8,9,10,...,243,244,245,246,247,248,249,250,251,999
Patient_ID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.894541,0.941105,0.900155,0.916167,0.910656,0.900076,0.906142,0.906315,0.894628,...,0.868549,0.890518,0.868256,0.857112,0.894977,0.844808,0.878832,0.911458,0.864998,0.237298
2,0.894541,1.000000,0.903770,0.865711,0.854922,0.827976,0.915758,0.902817,0.920410,0.808974,...,0.818879,0.872706,0.799682,0.815612,0.866371,0.791400,0.872935,0.876351,0.843309,0.262536
3,0.941105,0.903770,1.000000,0.937794,0.884078,0.903281,0.916944,0.939743,0.912112,0.860196,...,0.861377,0.878201,0.833596,0.869129,0.932881,0.828436,0.889868,0.909606,0.874682,0.297559
4,0.900155,0.865711,0.937794,1.000000,0.881771,0.902094,0.886630,0.905417,0.872123,0.888419,...,0.871267,0.846654,0.847257,0.871117,0.905814,0.803555,0.896448,0.857597,0.892029,0.216705
5,0.916167,0.854922,0.884078,0.881771,1.000000,0.939121,0.859263,0.875594,0.904124,0.949483,...,0.890029,0.902839,0.865302,0.897344,0.918664,0.869635,0.921692,0.925495,0.903866,0.299682


In [22]:
# 1. select the user we have selected Patient_ID 999 
#Get a list of Feature user already watched so we can remove from our recommendation list..

current_user_rating = dframe[(dframe.Patient_ID == 999) & (dframe.Value != 0)]['Feature']
current_user_rating = pd.DataFrame(current_user_rating, columns=['Feature'])

In [23]:
user_sim_df.shape

(252, 252)

In [24]:
# put similarity of current user i.e. 999 in a dataframe because later we need for weighted average..

curr_user_similarity = pd.DataFrame(user_sim_df.loc[999])

#just changing column name as similarity for better readability.
curr_user_similarity.rename(columns={999:'Similarity_Score'},inplace=True)
curr_user_similarity.sort_values(by='Similarity_Score', ascending=False, inplace=True)
curr_user_similarity1 = curr_user_similarity.iloc[1:, :]

# Reset index to move it to the first column
curr_user_similarity1.reset_index(inplace=True)

# Rename the index column
curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)

# Define a function to categorize values based on the condition
def categorize(x):
    if x < 165:
        return 'Patient'
    else:
        return 'Healthy'

# Apply the function to the first column of the dataframe
curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)

# Show the resulting dataframe
curr_user_similarity1.head()


C:\Users\system 4\AppData\Local\Temp\ipykernel_12832\4146630261.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1.rename(columns={'index': 'Index_Column'}, inplace=True)
C:\Users\system 4\AppData\Local\Temp\ipykernel_12832\4146630261.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_user_similarity1['Patient_Type'] = curr_user_similarity1['Patient_ID'].apply(categorize)


,Patient_ID,Similarity_Score,Patient_Type
0,135,0.466926,Patient
1,173,0.430724,Healthy
2,78,0.423274,Patient
3,97,0.421769,Patient
4,246,0.417758,Healthy


The Patient_ID most similar to the input user is showing, from 5 we can decide patient or healthy based on 
more number of healthy or patient among those top 5 list generated above.

In [25]:
similar_user = curr_user_similarity1.iloc[:5, :]
counts = similar_user['Patient_Type'].value_counts()
most_occurring_value = counts.index[0]
print("The user might fall under category of", most_occurring_value)

The user might fall under category of Patient


#As per the user category healthy or PD, we will give values to other unfilled features by that user

# Predicting Scoring of user: Top users of its category

In [26]:
select_simil_user = similar_user.loc[similar_user['Patient_Type'] == most_occurring_value]

In [27]:
df_simil = df1[df1['Patient_ID'].isin(select_simil_user['Patient_ID'])]
df_simil.head()

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
77,78,56.975065,0,1,9.55,5,0,1,1,1,...,46,0,2,43,45,43.750000,45,262,29,0
96,97,62.265514,1,2,15.69,7,0,1,0,2,...,38,2,0,35,35,22.500000,37,158,22,1
134,135,66.384280,0,2,12.16,7,0,1,0,1,...,49,2,1,49,48,41.669998,35,170,38,1


In [28]:
# Example DataFrame Y with some filled and some NaN values
X = df_simil
Y = pd.DataFrame(df3)

# Calculate the mean of each column in X excluding the first two columns
mean_cols = X.iloc[:, 2:].mean()

# Fill NaN values in DataFrame Y with the mean values
Y = Y.fillna(mean_cols)
Y.head(2)

,Patient_ID,Age,Anxiety,Apathy,Benton,Clock,Cognition,COGSTATE,Constipate,Depress,...,Semantic,SleepDay,SleepNight,STAIA,STAIS,Symbol_Digit,Trail_Making_A,Trail_Making_B,UPSIT,Urine
0,999,54.0,0.333333,2.0,12.466667,3.0,0.0,1.0,0.333333,1.333333,...,44.333333,1.333333,1.0,42.333333,42.666667,35.973333,39.0,196.666667,39.0,0.666667
